# Header

In [1]:
import os
import sys
from pathlib import Path

from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.llms import CTransformers
from langchain.prompts import ChatPromptTemplate

from rag_project.constants import (
    SENTENCE_TRANSFORMERS_MODEL_NAME, 
    BOOKS_CHROMA_DB_DIR, 
    BOOKS_COLLECTION_NAME, 
    LLAMA_MODEL_PATH
)

/home/lealdx/personal/rag_chat_project/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def load_local_llama() -> CTransformers:
    """Initialize and return a local LLM using CTransformers.""" 
    model_path = Path(LLAMA_MODEL_PATH)
    
    if not model_path.exists():
        raise FileNotFoundError(
            f"Model file not found at {model_path}. "
            "Please download it first using: \n"
            "make get_llama_model"
        )
    
    return CTransformers(
        model="TheBloke/Llama-2-7B-Chat-GGUF",
        model_file=str(model_path),
        model_type="llama",
        config={
            "max_new_tokens": 256,
            "temperature": 0.3,
            "top_k": 40,
            "top_p": 0.9,
            "repetition_penalty": 1.05,
            "stop": ["</s>", "[/INST]"],
            "context_length": 2048,
            "threads": min(4, os.cpu_count()),
            "seed": 42,
            "stream": True
        }
    )

In [3]:

def init_embeddings() -> HuggingFaceEmbeddings:
    """Initialize and return HuggingFace embeddings model."""
    model_name = SENTENCE_TRANSFORMERS_MODEL_NAME 
    model_kwargs = {"device": "cpu"}
    encode_kwargs = {
        "batch_size": 8,
        "normalize_embeddings": True,
        "convert_to_numpy": True
    }
    
    return HuggingFaceEmbeddings(
        model_name=model_name,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )

In [4]:
def init_chroma() -> Chroma:
    """Initialize and return Chroma DB."""
    embedding_function = init_embeddings()
    chroma_path = str(Path(BOOKS_CHROMA_DB_DIR))
    return Chroma(
        persist_directory=chroma_path,
        embedding_function=embedding_function,
        collection_name=BOOKS_COLLECTION_NAME
    )

# Load Objects

In [5]:
db = init_chroma()

In [6]:
model = load_local_llama()

Fetching 1 files: 100%|██████████| 1/1 [00:00<00:00,  1.82it/s]
Fetching 0 files: 0it [00:00, ?it/s]


# Prompt

In [7]:
PROMPT_TEMPLATE = """
[INST] <<SYS>>
You are a clear and concise assistant specialized in summarization and explanation.
Use only the provided context to answer the question.
If the context does not provide enough information, reply with:
"The context does not contain enough information to answer."
Avoid speculation.
<</SYS>>

Context:
{context}

Question:
{question}
[/INST]
"""


Test Queries:
```python
specific_queries = [
    "What does Alice drink or eat that makes her change size?",
    "Describe the Duchess's kitchen and what happens there",
    "What games are played in Wonderland?",
    "What poems or songs are recited in the story?",
    "How does Alice's adventure end?"
]
```

In [8]:
query_text = "Who is the White Rabbit and how does Alice first meet him?"

# Results

In [9]:

results = db.similarity_search_with_relevance_scores(query_text)
print(f"Results: {results}")

if len(results) == 0 or results[0][1] < 0.5:
    print(f"Unable to find matching results.")
else:
    print(f"Found {len(results)} relevant results")

Results: [(Document(id='6f9fa352-fadc-4f58-9357-86eebcb486cb', metadata={'start_index': 117652, 'source': '/home/marqsleal/Dev/Python/rag_project/data/books/alice_in_wonderland.md'}, page_content='So Alice began telling them her adventures from the time when she first saw the White Rabbit. She was a little nervous about it just at first, the two creatures got so close to her, one on each side, and opened their eyes and mouths so very wide, but she gained courage as she went on. Her listeners'), 0.7235418775839446), (Document(id='996fea39-7586-494e-a00a-b489f5b9801a', metadata={'source': '/home/marqsleal/Dev/Python/rag_project/data/books/alice_in_wonderland.md', 'start_index': 87027}, page_content='they were all ornamented with hearts. Next came the guests, mostly Kings and Queens, and among them Alice recognised the White Rabbit: it was talking in a hurried nervous manner, smiling at everything that was said, and went by without noticing her. Then followed the Knave of Hearts, carrying

In [10]:
context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, question=query_text)
print(prompt)

Human: 
[INST] <<SYS>>
You are a clear and concise assistant specialized in summarization and explanation.
Use only the provided context to answer the question.
If the context does not provide enough information, reply with:
"The context does not contain enough information to answer."
Avoid speculation.
<</SYS>>

Context:
So Alice began telling them her adventures from the time when she first saw the White Rabbit. She was a little nervous about it just at first, the two creatures got so close to her, one on each side, and opened their eyes and mouths so very wide, but she gained courage as she went on. Her listeners

---

they were all ornamented with hearts. Next came the guests, mostly Kings and Queens, and among them Alice recognised the White Rabbit: it was talking in a hurried nervous manner, smiling at everything that was said, and went by without noticing her. Then followed the Knave of Hearts, carrying the

---

Alice watched the White Rabbit as he fumbled over the list, feelin

In [11]:
response_text = model.invoke(prompt)

sources = [doc.metadata.get("source", None) for doc, _score in results]
formatted_response = f"Response: {response_text}\nSources: {sources}"
print(formatted_response)

Response: Based on the provided context, the answer to the question is:
The White Rabbit is a character who first appears in the story when he is seen by Alice as she is walking through a forest. He is described as being "a little nervous" and having "wide eyes and mouth." Alice gains courage as she continues to listen to the Rabbit's tale, and later, she recognizes him among a group of guests at a party, where he is talking in a "hurried nervous manner" and smiling at everything that is said.
Sources: ['/home/marqsleal/Dev/Python/rag_project/data/books/alice_in_wonderland.md', '/home/marqsleal/Dev/Python/rag_project/data/books/alice_in_wonderland.md', '/home/marqsleal/Dev/Python/rag_project/data/books/alice_in_wonderland.md', '/home/marqsleal/Dev/Python/rag_project/data/books/alice_in_wonderland.md']
